In [26]:
#@title
import pandas as pd
import numpy as np
import os

# Mount the drive
from google.colab import drive
drive.mount('/content/drive')

from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())

# List the tab names. Could be used for asserting in next section
# titles_list = gc.open('Copy of Hongwei Dong lab case tracker.xlsx').worksheets()
# print(titles_list)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# New Section

In [30]:
# tab names and pivotal column (case column)
tab_names = {'U01 ':2,'U19 Salk Institute':2,'U19 CSHL':2,'MCP':1,'Basal Ganglia':2,'RF1 (HPF)':2,'BLA project':2}
tabs = {}
for k,v in tab_names.items():
  worksheet = gc.open('Copy of Hongwei Dong lab case tracker.xlsx').worksheet(k)
  tabs[k] = pd.DataFrame(worksheet.get_all_values())  

distinctCases = pd.DataFrame()
tracersPathways = pd.DataFrame()
actualInjections = pd.DataFrame()

# ToDo: Please revist the parsing
for k,tab in tabs.items():  
  piv = tab_names[k]-1
  cases = pd.DataFrame(tab.iloc[:, piv].unique()) # Distinct cases  
  cases = cases.applymap(str)
  cases = cases.loc[cases[0].str.startswith('S')]
  cases['length']= (cases[0]).str.len()
  cases = cases[(cases.length>=11) & (cases.length<=13)] # some cases have asterisks?
  tracers = pd.DataFrame(tab.iloc[:, piv+3]) # total
  injections = pd.DataFrame(tab.iloc[:, piv+4]) # total

  distinctCases = distinctCases.append(cases)
  tracersPathways = tracersPathways.append(tracers)
  actualInjections = actualInjections.append(injections)

tracersPathways = tracersPathways.dropna()
actualInjections = actualInjections.dropna()
distinctCases = distinctCases.dropna()

distinctCases = pd.DataFrame(distinctCases.iloc[:,0].unique())
distinctCases.index = distinctCases.index + 1
distinctCases.to_csv('/distinctCases.csv', header=False)
print('/distinctCases.csv')

tracersPathways = tracersPathways.iloc[:, 0]
tracersPathways.index = tracersPathways.index + 1
pd.DataFrame(tracersPathways).to_csv('/tracersPathways.csv', header=False)
print('/tracersPathways.csv')

pd.DataFrame(actualInjections.iloc[:, 0]).to_csv('/actualInjections.csv')
print('/actualInjections.csv')

/distinctCases.csv
/tracersPathways.csv
/actualInjections.csv
